In [1]:
import os
import pandas as pd
import numpy as np
from sim.core.cascade import RepoCascade
from sim.core.inputs import load_inputs, load_k_covid
from sim.ind import ModelPupr, get_intv

In [2]:
iso = 'IND'
in_folder = f'results/B_{iso}'
out_folder = f'results/I_{iso}'

In [3]:
os.makedirs(out_folder, exist_ok=True)

In [4]:
pars_cs = pd.read_csv(f'results/pars_{iso}.csv')
repo_cs = RepoCascade(pars_cs, 2010)
k_covid = load_k_covid(f'{in_folder}/k_covid.csv')

inp = load_inputs(f'data/pars/{iso}')
model = ModelPupr(inp)

In [5]:
particles = pd.read_csv(f'{in_folder}/Post.csv')
particles = [dict(row) for _, row in particles.iterrows()]

particles = particles

In [6]:
particles[0]

{'Unnamed: 0.1': 0,
 'Unnamed: 0': 0,
 'beta': 9.644981920003955,
 'rr_sus_slat': 0.467021082426154,
 'rr_sus_rec': 0.1642372541134697,
 'p_primary': 0.1001911979234508,
 'r_clear': 0.0259534666547411,
 'r_sc': 0.2241997374367901,
 'adr': 0.029397969588018,
 'rr_inf_asym': 1,
 'rr_inf_cs': 1,
 'r_lat': 0.5,
 'r_stab': 0.6666666666666666,
 'r_react': 0.001,
 'r_relapse': 0.001,
 'r_relapse_td': 0.008,
 'r_relapse_tc': 0.004,
 'p_cs_pub': 0.483,
 'p_dx_pub': 0.65,
 'p_dx_pri': 0.27,
 'dur_succ': 0.5,
 'r_act': 0.0556736040435661,
 'cas': 'Cascade(1.060 -> 2.889(45%) -> 3.482(45%))',
 'k_covid': '<scipy.interpolate.interpolate.interp1d object at 0x000001323D2DEA90>'}

In [7]:
baseline = list()
for i, p in enumerate(particles):
    p['cas'] = repo_cs.sample()
    p['k_covid'] = k_covid
    p['rt_cs'] = 0.041235603
    ys, ms, _ = model.simulate_to_fit(p, t_eval=np.linspace(2015, 2022, 8))
    baseline.append((p, ys, ms.assign(Key = i, Scenario = 'Baseline')))

In [8]:
pr_hi = 0.16
rat = 3

pr_prev_hi = pr_hi * rat / (pr_hi * rat + (1 - pr_hi))

In [9]:
rd0 = 2
scs = {
    'ACF2_b': lambda p: get_intv(p, rd_csi=rd0, rd_recsi=rd0),
    'Dx90_ACF2_b': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9, rd_csi=rd0, rd_recsi=rd0),
    'PPM90_ACF2_b': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9, ppm=0.9, 
                                     rd_csi=rd0, rd_recsi=rd0, r_asym_acf=pr_prev_hi * 0.1)
}

rd1 = 2 / 0.45 * 0.8
scs.update({
    'ACF2_cxr': lambda p: get_intv(p, rd_csi=rd1, rd_recsi=rd1),
    'Dx90_ACF2_cxr': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9, rd_csi=rd1, rd_recsi=rd1),
    'PPM90_ACF2_cxr': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9, ppm=0.9, 
                                     rd_csi=rd1, rd_recsi=rd1, r_asym_acf=pr_prev_hi * 0.1)
})

rd2 = 2 / 0.45 * 0.95
scs.update({
    'ACF2_xpert': lambda p: get_intv(p, rd_csi=rd2, rd_recsi=rd2),
    'Dx90_ACF2_xpert': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9, rd_csi=rd2, rd_recsi=rd2),
    'PPM90_ACF2_xpert': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9, ppm=0.9, 
                                     rd_csi=rd2, rd_recsi=rd2, r_asym_acf=pr_prev_hi * 0.1)
})



intvs = list()

for i, (p, ys, _) in enumerate(baseline):
    _, ms0, _ = model.simulate_intv(p, ys)
    intvs.append(ms0.assign(Key = i, Scenario = 'Baseline'))
    
    for k, sc in scs.items():
        intv = sc(p)
        _, ms1, _ = model.simulate_intv(p, ys, intv)
        intvs.append(ms1.assign(Key = i, Scenario = k))

intvs = pd.concat(intvs)

In [10]:
intvs

,IncR_apx,MorR_apx,CNR_apx,AcfR,AcfRAsym,CumInc,CumMor,CumACF,IncR,MorR,DetR,CNR,PrLat,PrRecent,Prev,PrevA,PrevS,PrevC,Key,Scenario
Time,,,,,,,,,,,,,,,,,,,,
2022.0,0.003332,0.000451,0.003343,0.000000,0.000000,1.387784e+09,1.885542e+08,0.000000e+00,0.003338,0.000458,0.002042,0.002980,0.507431,0.860706,0.005274,0.002510,0.001147,0.000868,0,Baseline
2023.0,0.003326,0.000426,0.003828,0.000000,0.000000,1.392491e+09,1.891749e+08,0.000000e+00,0.003333,0.000422,0.002404,0.003655,0.507651,0.859960,0.005121,0.002538,0.000911,0.000797,0,Baseline
2024.0,0.003233,0.000391,0.003305,0.000000,0.000000,1.397164e+09,1.897529e+08,0.000000e+00,0.003232,0.000390,0.002130,0.003319,0.506642,0.855063,0.004822,0.002511,0.000803,0.000675,0,Baseline
2025.0,0.003137,0.000376,0.003166,0.000000,0.000000,1.401736e+09,1.903022e+08,0.000000e+00,0.003136,0.000375,0.002050,0.003233,0.504947,0.850660,0.004651,0.002449,0.000782,0.000636,0,Baseline
2026.0,0.003055,0.000365,0.003076,0.000000,0.000000,1.406219e+09,1.908383e+08,0.000000e+00,0.003054,0.000365,0.001992,0.003160,0.502918,0.846886,0.004520,0.002384,0.000762,0.000617,0,Baseline
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2031.0,0.002271,0.000203,0.001270,0.001032,0.000068,2.121026e+09,2.975022e+08,1.742481e+07,0.002268,0.000202,0.000827,0.001311,0.462189,0.802333,0.002938,0.001865,0.000333,0.000019,299,PPM90_ACF2_xpert
2032.0,0.002125,0.000188,0.001180,0.000959,0.000063,2.124384e+09,2.978010e+08,1.894616e+07,0.002123,0.000188,0.000768,0.001219,0.450535,0.793857,0.002731,0.001736,0.000309,0.000017,299,PPM90_ACF2_xpert
2033.0,0.002001,0.000176,0.001104,0.000898,0.000059,2.127558e+09,2.980817e+08,2.037436e+07,0.002000,0.000176,0.000719,0.001141,0.439028,0.786524,0.002556,0.001626,0.000289,0.000016,299,PPM90_ACF2_xpert


In [11]:
intvs.to_csv(f'{out_folder}/RunSens.csv')